In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("tweets2020.csv")

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,2,Classificação
0,0,0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,10
1,1,1,@ShakedownVail Yo. What a #kickass #chill bar ...,10
2,2,2,"RT @grazianopascale: ""Parece la trama de un fi...",10
3,3,3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,10
4,4,4,RT @NetoItirussu: Hoje em depoimento da operaç...,10


In [6]:
df = df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)

In [7]:
df.head()

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,10
1,@ShakedownVail Yo. What a #kickass #chill bar ...,10
2,"RT @grazianopascale: ""Parece la trama de un fi...",10
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,10
4,RT @NetoItirussu: Hoje em depoimento da operaç...,10


### Link pra uso do bard:
https://ai.google.dev/gemini-api/docs/get-started/python?hl=pt-br

In [8]:
pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/media/marcos/2caebd18-4175-48de-967d-932bd88604d32/Ajuda Mestrado/tweets/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [12]:
model = genai.GenerativeModel('gemini-pro')

In [13]:
def classify_tweet(tweet):
    question = "Estou classificando tweets em português brasileiro sobre a bolsa de valores do brasil. Quero que você classifique esse tweet em sentimentos onde cada número é um sentimento: 0- neutro, 1 - positivo, 2- negativo. Esse é o tweet: "+tweet+". Me mande apenas o número da classificação"
    response = model.generate_content(question)
    response = response.text
    for word in response.split():
        if word == '0' or word == '1' or word == '2':
            continue
        else:
            return None
        
    return response


In [14]:
df.head()

,2,Classificação
0,RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...,10
1,@ShakedownVail Yo. What a #kickass #chill bar ...,10
2,"RT @grazianopascale: ""Parece la trama de un fi...",10
3,RT @marcofeliciano: Pare e compare: PETROBRAS ...,10
4,RT @NetoItirussu: Hoje em depoimento da operaç...,10


In [16]:
last_index = 90
count = 0

indices = df.index[df.index >= last_index]

for index in indices:    
    try:
        if count > 100:
            break
        else:
            if df.loc[index,'Classificação'] == 10: 
                print(index)  
                tweet = df.loc[index,'2']            
                response = classify_tweet(tweet)
                
                if response == None:
                    continue
            
                else:
                    df.loc[index,'Classificação'] = int(response)
                    count += 1
            
            else: continue
            
    except:
        continue

90
92
93
94
95
96
97
98
99
100
101
102
103
104
106
107
108
109
110
111
112
113
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230


In [23]:
def rate_tweets(df, last_index):
    indices = df.index[df.index >= last_index]

    for index in indices:
        print(df.loc[index,:])
        answer = int(input("Coloque 0 pra continuar, 1 pra alterar classificação e 2 pra parar: "))
        if answer == 0:
            continue
        elif answer == 1:
            right_answer = int(input("Coloque a classicação certa: "))
            df.loc[index,"Classificação"] = right_answer
            print(len(df[df["Classificação"]]))
        else:
            return index

In [24]:
rate_tweets(df,0)

Unnamed: 0                                                       0
2                RT @AjudemSerena: IMPLORO POR SOCORRO!\nPor fa...
Classificação                                                   10
Name: 0, dtype: object


0

In [27]:
df.to_csv("tweets2020.csv")